In [1]:
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from io import StringIO
from selenium.webdriver.chrome.options import Options
import pandas as pd
from datetime import datetime
import os
import glob

In [2]:
start_time = datetime.now()

In [3]:
dir = "Y:\\delete\\US National Toxicolgy Program"
for f in os.listdir(dir):
    os.remove(os.path.join(dir, f)) 

In [4]:
ser = Service("C:/Users/AIMPAI0/Downloads/chromedriver.exe")
op = webdriver.ChromeOptions()
prefs = {"download.default_directory" : "Y:\\delete\\US National Toxicolgy Program"}
op.add_experimental_option("prefs",prefs)
driver = webdriver.Chrome(service=ser, options=op)
driver.maximize_window()  
driver.get("https://ntp.niehs.nih.gov/whatwestudy/assessments/cancer/roc/index.html")
driver.implicitly_wait(5)
driver.find_element(by=By.XPATH, value='//*[@id="rocAccessListingsBox"]/ul/li[7]/a').click()
time.sleep(10)
driver.close()

In [17]:
path = 'Y:\\delete\\US National Toxicolgy Program'
filenames = glob.glob(path + "\*.xlsx")
for f in filenames:
    dataset = pd.read_excel(f,sheet_name='CAS #s')
dataset = dataset[dataset['CASRN'].notna()]
dataset.reset_index(inplace = True, drop = True)
dataset.columns.values[2] = "Group"
dataset.rename(columns = {'CASRN':'CAS Number'}, inplace = True)
dataset = dataset.applymap(lambda x: x.strip() if isinstance(x, str) else x)
CAS = pd.read_excel('Y:/delete/Chemical CAS.xlsx')
CAS['value'] = 'Unknown'
CAS = CAS.applymap(lambda x: x.strip() if isinstance(x, str) else x)

result = pd.merge(dataset, CAS, how="outer", on="CAS Number")
result.Group.fillna(result.value, inplace=True)
del result['value']
result.head(2)

d:\python3.9\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,CAS Number,NAME OR SYNONYM,Group
0,50-00-0,Formaldehyde,Known
1,50-18-0,Cyclophosphamide,Known


In [18]:
result['Jurisdiction'] = 'US'
result['RegulatoryAgency'] = 'National Toxicology Program (NTP)'
result['TypeOfRegulation'] = 'Health'
result['Criteria'] = 'Carcinogenicity'


result['Description'] = 'May cause cancer'
result.loc[result['Group'] == 'RAHC', 'Description'] = 'Suspected of Causing Cancer'
result.loc[result['Group'] == 'Unknown', 'Description'] = 'Not evaluated'


result['HazardClassification'] = 'High'
result.loc[result['Group'] == 'RAHC', 'HazardClassification'] = 'Moderate'
result.loc[result['Group'] == 'Unknown', 'HazardClassification'] = 'Low'

result['CauseOfClassification'] = result['Group']

result['SourceName'] = 'International Agency for Research on Cancer'
result['SourceURL'] = 'https://monographs.iarc.who.int/list-of-classifications'

result = result.drop(['Group','NAME OR SYNONYM'], axis=1)
result.rename(columns = {'CASRN':'CAS Number'}, inplace = True)

result.head(2)

,CAS Number,Jurisdiction,RegulatoryAgency,TypeOfRegulation,Criteria,Description,HazardClassification,CauseOfClassification,SourceName,SourceURL
0,50-00-0,US,National Toxicology Program (NTP),Health,Carcinogenicity,May cause cancer,High,Known,International Agency for Research on Cancer,https://monographs.iarc.who.int/list-of-classi...
1,50-18-0,US,National Toxicology Program (NTP),Health,Carcinogenicity,May cause cancer,High,Known,International Agency for Research on Cancer,https://monographs.iarc.who.int/list-of-classi...


In [ ]:
result.to_csv('Y:/delete/Reugulatory Assessment.csv',index=False,mode='a',header=False)

In [37]:
end_time = datetime.now()
print('The total time taken is', (end_time - start_time))

The total time taken is 0:31:17.877340
